In [24]:
'''
Author: Anna Diachenko
Date: 5/25/2021
Project Description: Web Scraping a Wikipedia table and requesting data from an API 
                     using the values from a column in a data frame 
'''


# Importing libraries
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [ ]:
# Scraping the table with Academy Award-winning films
url = "https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films"

page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

print(soup.prettify())

all_tables = soup.findAll('table')
all_tables[0]

right_table = soup.find('table', class_='wikitable sortable') 
right_table
print(right_table)

In [26]:
# Creating lists for as each column in the table
Film = []
Year = []
Awards = []
Nominations = []

for row in right_table.findAll('tr'):
  cells = row.findAll('td')
  if len(cells)== 4:
    Film.append(cells[0].text.strip())  
    Year.append(cells[1].text.strip()) 
    Awards.append(cells[2].text.strip()) 
    Nominations.append(cells[3].text.strip()) 

In [27]:
# Creating the database with the movies

wiki_films = {
                'Film': Film,
                'Year': Year,
                'Awards': Awards,
                'Nominations': Nominations
             } 

df1 = pd.DataFrame(wiki_films)

# Selecting first 31 films (15 nominees in 2020/21 and 16 nominees in 2019)
df1 = df1.iloc[0:31]
df1

,Film,Year,Awards,Nominations
0,Nomadland,2020/21,3,6
1,The Father,2020/21,2,6
2,Judas and the Black Messiah,2020/21,2,6
3,Minari,2020/21,1,6
4,Mank,2020/21,2,10
5,Sound of Metal,2020/21,2,6
6,Ma Rainey's Black Bottom,2020/21,2,5
7,Promising Young Woman,2020/21,1,5
8,Tenet,2020/21,1,2
9,Soul,2020/21,2,3


In [28]:
# Converting Awards and Nominations columns to floats to produce the statistics later in the program
df1['Awards'] = pd.to_numeric(df1['Awards'],downcast='float')
df1['Nominations'] = pd.to_numeric(df1['Nominations'],downcast='float')

In [29]:
# Creating a list from the Film column that will make requests to the API
movies = df1['Film'].tolist()

In [ ]:
# Assigning the OMDb API key
key = "c46cddd2"

# Creating lists to store data
Title = []
Country = []
Genre = []
BoxOffice = []
imdbRating = []
Runtime = []

# Creating a loop and specifying the URL to make a request to the API
for movie in movies:
  url = "http://www.omdbapi.com/?t="+movie+"&apikey="+key
  print(movie)
  r = requests.get(url)
  
  # Using If statement to make sure that the requests were processed successfully (Status code = 200)
  if(r.status_code == 200):

    # Retriving the JSON sent by the API
    data = r.json()

    # We can use the line below to display the JSON output from the API if needed (removing #)
    #print(json.dumps(data,sort_keys=True,indent=5))

    # Adding data to designated lists
    Title.append(data['Title'])
    print("Title: "+str(data['Title']))

    Country.append(data['Country'])
    print("Country: "+str(data['Country']))
    
    Genre.append(data['Genre'])
    print("Genre: "+str(data['Genre']))

    BoxOffice.append(data['BoxOffice'])
    print("US Box Office: "+str(data['BoxOffice']))

    imdbRating.append(data['imdbRating'])
    print("IMDb Rating: "+str(data['imdbRating']))

    Runtime.append(data['Runtime'])
    print("Runtime: "+str(data['Runtime'])+'\n')

In [ ]:
# Creating a dictionary which will be converted to a dataframe
award_films = {
                'Film': Title,
                'Country': Country,
                'Genre': Genre,
                'Box Office': BoxOffice,
                'IMDb Rating': imdbRating,
                'Runtime': Runtime
   
              }


df2 = pd.DataFrame(award_films)

# Removing all strings and NaNs before converting pandas columns to a numeric type
# Removing the NaNs if any exists
df2.dropna(inplace=True)

# Removing the 'N/A' by selecting all the rows from 'IMDB rating' which don't have NA and reassigning such to df2
df2 = df2.loc[df2['IMDb Rating'] != 'N/A']

# Converting the IMDb Rating column to float
df2['IMDb Rating'] = pd.to_numeric(df2['IMDb Rating'], downcast = 'float')

# Then removing 'the min from the runtime column and replace it with an empty string
df2['Runtime'] = df2['Runtime'].str.replace('min','',)

# Now converting it to float
df2['Runtime'] = pd.to_numeric(df2['Runtime'], downcast = 'float')

# Removing the N/A rows from Box Office Column
df2= df2.loc[df2['Box Office'] != 'N/A']
# Replacing the commas with an empty string
df2['Box Office'] = df2['Box Office'].str.replace(',','')
# Replacing the dollar sign with an empty string 
df2['Box Office'] = df2['Box Office'].str.replace('$','')
# And converting it to float
df2['Box Office'] = pd.to_numeric(df2['Box Office'], downcast= 'float')

# Displaying the dataframe to see the changes made
df2

In [32]:
# Using the difference for columns from df2 which are not in df1 to avoid repetition during the merge
df2 = df2[df2.columns.difference(df1.columns)]

# Merging two dataframes horizontally 
df3 = pd.concat([df1, df2], axis=1)

# Removing the NaNs 
df3.dropna(inplace=True)

# Removing the 'N/A' by selecting all the rows from 'IMDB rating' which don't have NA and reassigning such to df2
df3 = df3.loc[df3['IMDb Rating'] != 'N/A']
df3= df3.loc[df3['Box Office'] != 'N/A']
df3= df3.loc[df3['Runtime'] != 'N/A']

# Converting these columns to float
df3['IMDb Rating'] = pd.to_numeric(df3['IMDb Rating'], downcast = 'float')
df3['Box Office'] = pd.to_numeric(df3['Box Office'], downcast= 'float')
df3['Runtime'] = pd.to_numeric(df3['Runtime'], downcast = 'float')

df3

,Film,Year,Awards,Nominations,Box Office,Country,Genre,IMDb Rating,Runtime
0,Nomadland,2020/21,3.0,6.0,2143000.0,"USA, Germany",Drama,7.4,107.0
1,The Father,2020/21,2.0,6.0,2064299.0,"UK, France",Drama,8.3,97.0
2,Judas and the Black Messiah,2020/21,2.0,6.0,5446607.0,USA,"Biography, Drama, History",7.5,126.0
3,Minari,2020/21,1.0,6.0,2964816.0,USA,Drama,7.5,115.0
7,Promising Young Woman,2020/21,1.0,5.0,6460965.0,"UK, USA","Crime, Drama, Thriller",7.5,113.0
8,Tenet,2020/21,1.0,2.0,58456624.0,"UK, USA","Action, Sci-Fi, Thriller",7.4,150.0
12,Colette,2020/21,1.0,1.0,5137622.0,"UK, USA, France, Hungary, Netherlands","Biography, Drama, History, Romance",6.7,111.0
15,Parasite,2019,4.0,6.0,53369748.0,South Korea,"Comedy, Drama, Thriller",8.6,132.0
16,Ford v Ferrari,2019,2.0,4.0,117624360.0,USA,"Action, Biography, Drama, Sport",8.1,152.0
18,The Neighbors' Window,2019,1.0,1.0,330661.0,USA,"Short, Drama",7.3,21.0


In [ ]:
# Displaying the statistics
df3.describe()

In [34]:
# Exporting the combined dataframe in csv file
df3.to_csv("Academy Award-winning films 2019-2021.csv")